In [1]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# to visualize more columns and rows
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# reading txn data
txn_data = pd.read_csv('interview_txn_data.csv')

# reading ticket data
ticket_data = pd.read_csv('interview_ticket_data.csv')

In [3]:
# inspecting txn data

In [4]:
txn_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
0,2.0,12104,2018-10-14 13:52:31 UTC,2,1,1
1,2.0,12104,2018-08-11 16:55:33 UTC,2,1,1
2,2.0,24979501,2019-02-21 16:33:03 UTC,2,1,1
3,2.0,12104,2018-09-20 18:15:22 UTC,2,1,1
4,100.0,15957319,2018-07-22 07:14:24 UTC,2,1,1


In [5]:
ticket_data.head()

,TicketCreatedDateTime,AgentRefID
0,2018-11-12 15:27:45,3
1,2018-11-12 15:29:26,3
2,2018-09-19 12:23:06,4
3,2018-08-30 12:18:25,4
4,2018-09-14 12:55:09,4


In [6]:
ticket_data.shape

(25574, 2)

In [23]:
# converting date columns into datetime datatypes
txn_data['CreatedDate'] = pd.to_datetime(txn_data['CreatedDate'])
ticket_data['TicketCreatedDateTime'] = pd.to_datetime(ticket_data['TicketCreatedDateTime'])

In [22]:
txn_data['CreatedDate'] = pd.to_datetime(txn_data['CreatedDate'].dt.date)
ticket_data['TicketCreatedDateTime'] = pd.to_datetime(ticket_data['TicketCreatedDateTime'].dt.date)

AttributeError: Can only use .dt accessor with datetimelike values

In [25]:
txn_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
0,2.0,12104,2018-10-14,2,1,1
1,2.0,12104,2018-08-11,2,1,1
2,2.0,24979501,2019-02-21,2,1,1
3,2.0,12104,2018-09-20,2,1,1
4,100.0,15957319,2018-07-22,2,1,1


## Data Preparation

    - Data preparation is done till 21 Jan 2019
    - For lable creation of Churn or not churn, Data from 22 Jan 2019 to 22 Feb 2019 will be used

In [26]:
# data preparation till 22 Jan 2019
# for label Creation

label_data = txn_data[txn_data.CreatedDate >= '2019-01-22']
label_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
2,2.0,24979501,2019-02-21,2,1,1
10,2.0,12104,2019-02-20,2,1,1
13,2.0,12104,2019-02-18,2,1,1
19,1.0,0,2019-02-18,2,1,1
22,1.0,0,2019-02-21,2,1,1


In [27]:
# For Data Preparation till 21 Jan 2019
txn_data = txn_data[txn_data.CreatedDate < '2019-01-22']
ticket_data = ticket_data[ticket_data.TicketCreatedDateTime < '2019-01-22']

In [28]:
# Feature Creation from transaction Data

In [29]:
txn_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
0,2.0,12104,2018-10-14,2,1,1
1,2.0,12104,2018-08-11,2,1,1
3,2.0,12104,2018-09-20,2,1,1
4,100.0,15957319,2018-07-22,2,1,1
5,2.0,12104,2018-10-15,2,1,1


In [30]:
txn_data['TransactionStatusRefID'].value_counts()

2     2814307
6      115094
9       25088
10      21733
1       10490
Name: TransactionStatusRefID, dtype: int64

In [31]:
# checking missing values
def missing(df):
    print (round((df.isnull().sum() * 100/ len(df)),2).sort_values(ascending=False))

missing(txn_data)

DistributorRefID          0.0
AgentRefID                0.0
TransactionStatusRefID    0.0
CreatedDate               0.0
PayeeRefID                0.0
Amount                    0.0
dtype: float64


In [32]:
# No missing values

In [33]:
# Features from TransactionStatusRefID columns
def get_consolidated(customer_details):
    total_payment = len(customer_details)
    total_successfull_payments = len(customer_details[customer_details["TransactionStatusRefID"]==2])
    total_failed_payments = len(customer_details[customer_details["TransactionStatusRefID"]!=2])
    return total_payment, total_successfull_payments, total_failed_payments

In [34]:
txn_data_group= txn_data.groupby("AgentRefID", sort=False).apply(lambda cust: get_consolidated(cust))

In [35]:
txn_data_group = txn_data_group.reset_index()
txn_data_group[['total_payment', 'total_successfull_payments', 'total_failed_payments']] = pd.DataFrame(txn_data_group[0].tolist(), index=txn_data_group.index)
txn_data_group = txn_data_group.drop(0, axis=1)

In [36]:
txn_data_group.head()

,AgentRefID,total_payment,total_successfull_payments,total_failed_payments
0,1,22,21,1
1,55,132,128,4
2,83,3361,3253,108
3,180,1965,1878,87
4,29,901,843,58


In [37]:
txn_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
0,2.0,12104,2018-10-14,2,1,1
1,2.0,12104,2018-08-11,2,1,1
3,2.0,12104,2018-09-20,2,1,1
4,100.0,15957319,2018-07-22,2,1,1
5,2.0,12104,2018-10-15,2,1,1


In [40]:
# Features created from amount column
def get_consolidated_amount(customer_details):
    # only succesfull payment filter
    total_successful_payments_df  = customer_details[customer_details["TransactionStatusRefID"]==2]
    # total amount transactor
    total_amount_transaction = sum(total_successful_payments_df["Amount"])
    
    #max, min, median and aveage amount transaction
    max_amount_transaction = max(total_successful_payments_df["Amount"], default =0)
    min_amount_transaction = min(total_successful_payments_df["Amount"], default =0)
    avg_amount_transaction = total_successful_payments_df["Amount"].mean()
    median_amount_transaction = total_successful_payments_df["Amount"].median()
    
    total_amount_transaction_gt100 = sum(total_successful_payments_df[total_successful_payments_df["Amount"]>100]['Amount'])
    total_count_transaction_gt100 = len(total_successful_payments_df[total_successful_payments_df["Amount"]>100]['Amount'])
    
    total_amount_transaction_gt1000 = sum(total_successful_payments_df[total_successful_payments_df["Amount"]>1000]['Amount'])
    total_count_transaction_gt1000 = len(total_successful_payments_df[total_successful_payments_df["Amount"]>1000]['Amount'])
    
    # PayeeRefId==0 is considerd as amount received as it is end customer but if it's 0 means money has credited
    total_amount_transcation_debit = sum(total_successful_payments_df[total_successful_payments_df["PayeeRefID"]!=0]['Amount'])
    total_amount_transaction_credit =sum(total_successful_payments_df[total_successful_payments_df["PayeeRefID"]==0]['Amount'])
    
    
    
    return total_amount_transaction,max_amount_transaction,min_amount_transaction,avg_amount_transaction,median_amount_transaction, total_amount_transaction_gt100, total_count_transaction_gt100, total_amount_transaction_gt1000,total_count_transaction_gt1000, total_amount_transcation_debit, total_amount_transaction_credit

In [41]:
txn_data_group_amt= txn_data.groupby("AgentRefID", sort=False).apply(lambda cust: get_consolidated_amount(cust))

In [42]:
txn_data_group_amt = txn_data_group_amt.reset_index()
txn_data_group_amt[['total_amount_transaction', 'max_amount_transaction','min_amount_transaction','avg_amount_transaction','median_amount_transaction','total_amount_transaction_gt100', 'total_count_transaction_gt100', 'total_amount_transaction_gt1000','total_count_transaction_gt1000', 'total_amount_transcation_debit', 'total_amount_transaction_credit']] = pd.DataFrame(txn_data_group_amt[0].tolist(), index=txn_data_group_amt.index)
txn_data_group_amt = txn_data_group_amt.drop(0, axis=1)

In [43]:
txn_data_group_amt.head()

,AgentRefID,total_amount_transaction,max_amount_transaction,min_amount_transaction,avg_amount_transaction,median_amount_transaction,total_amount_transaction_gt100,total_count_transaction_gt100,total_amount_transaction_gt1000,total_count_transaction_gt1000,total_amount_transcation_debit,total_amount_transaction_credit
0,1,246.0,100.0,1.0,11.714286,2.0,0.0,0,0.0,0,238.0,8.0
1,55,1074317.0,25000.0,1.0,8393.101562,4999.0,1074274.0,85,1069574.0,79,1074274.0,43.0
2,83,7335134.0,25000.0,1.0,2254.882877,1950.0,7333954.0,2320,6879216.0,1797,7334204.0,930.0
3,180,5929296.0,25000.0,1.0,3157.239617,2000.0,5929067.0,1847,5508229.0,1372,5929274.0,22.0
4,29,1891118.0,25000.0,1.0,2243.319098,2000.0,1890853.0,677,1799222.0,551,1890954.0,164.0


In [57]:
# Features from PayRefID and CreatedDate
# Features created from amount column
def get_consolidated_date(customer_details):
    # only succesfull payment filter
    total_successful_payments_df  = customer_details[customer_details["TransactionStatusRefID"]==2]
    total_successful_payments_df['Year'] = total_successful_payments_df["CreatedDate"].dt.year
    total_successful_payments_df['Month'] = total_successful_payments_df["CreatedDate"].dt.month
    total_successful_payments_df['Days'] = total_successful_payments_df["CreatedDate"].dt.day
    
    # total unique payee
    total_unique_payee = total_successful_payments_df["PayeeRefID"].nunique()
    
    # total unique distributer
    total_unique_distributer = total_successful_payments_df["DistributorRefID"].nunique()
    
    #first_transaction_date = 
    first_txn_date = min(total_successful_payments_df["CreatedDate"],default="")
    last_payment_date = max(total_successful_payments_df["CreatedDate"],default="")
    # aveage use of payment per month
    avg_count_payment_permonth = (total_successful_payments_df.groupby(['Year', "Month"])["AgentRefID"].count()).median()
    avg_count_payment_gt100_permonth =(total_successful_payments_df[total_successful_payments_df['Amount']>=100].groupby(['Year', "Month"])["AgentRefID"].count()).mean()
    
    #average amount transact per month
    avg_amount_payment_permonth = total_successful_payments_df.groupby(['Year', "Month"])["Amount"].mean().reset_index()["Amount"].mean()
    avg_amout_payment_gt100_permonth =total_successful_payments_df[total_successful_payments_df['Amount']>=100].groupby(['Year', "Month"])["Amount"].mean().reset_index()["Amount"].mean()
    
    # last month 
    last_month_amount_txn = total_successful_payments_df[(total_successful_payments_df.Year ==2019) &(total_successful_payments_df.Month >=1)]["Amount"].sum()
    last_month_amount_gt100 =total_successful_payments_df[(total_successful_payments_df.Year ==2019) &(total_successful_payments_df.Month >=1) & (total_successful_payments_df['Amount']>=100)]["Amount"].sum()
    
    #txn first week of each month
    avg_txn_first_week= total_successful_payments_df[total_successful_payments_df.Days <=7]["Amount"].mean()
    avg_last_month_1week_amount_txn = total_successful_payments_df[(total_successful_payments_df.Year ==2019) &(total_successful_payments_df.Month >=1) & (total_successful_payments_df.Days <=7)]["Amount"].sum()
    
    
    return total_unique_payee,total_unique_distributer, first_txn_date ,last_payment_date ,avg_count_payment_permonth ,avg_count_payment_gt100_permonth,avg_amount_payment_permonth , avg_amout_payment_gt100_permonth, last_month_amount_txn,avg_txn_first_week, avg_last_month_1week_amount_txn,last_month_amount_gt100


In [58]:
txn_data_group_date= txn_data.groupby("AgentRefID", sort=False).apply(lambda cust: get_consolidated_date(cust))

In [59]:
txn_data_group_date = txn_data_group_date.reset_index()
txn_data_group_date[['total_unique_payee', 'total_unique_distributer','first_txn_date' ,'last_payment_date' ,'avg_count_payment_permonth' ,'avg_count_payment_gt100_permonth','avg_amount_payment_permonth' , 'avg_amout_payment_gt100_permonth', 'avg_last_month_amount_txn','avg_txn_first_week', 'avg_last_month_1week_amount_txn','last_month_amount_gt100']] = pd.DataFrame(txn_data_group_date[0].tolist(), index=txn_data_group_date.index)
txn_data_group_date = txn_data_group_date.drop(0, axis=1)

In [60]:
txn_data_group_date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 13 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   AgentRefID                        4560 non-null   int64         
 1   total_unique_payee                4560 non-null   int64         
 2   total_unique_distributer          4560 non-null   int64         
 3   first_txn_date                    4550 non-null   datetime64[ns]
 4   last_payment_date                 4550 non-null   datetime64[ns]
 5   avg_count_payment_permonth        4550 non-null   float64       
 6   avg_count_payment_gt100_permonth  4460 non-null   float64       
 7   avg_amount_payment_permonth       4550 non-null   float64       
 8   avg_amout_payment_gt100_permonth  4460 non-null   float64       
 9   avg_last_month_amount_txn         4560 non-null   float64       
 10  avg_txn_first_week                3913 non-null 

In [48]:
txn_data.head()

,Amount,PayeeRefID,CreatedDate,TransactionStatusRefID,AgentRefID,DistributorRefID
0,2.0,12104,2018-10-14,2,1,1
1,2.0,12104,2018-08-11,2,1,1
3,2.0,12104,2018-09-20,2,1,1
4,100.0,15957319,2018-07-22,2,1,1
5,2.0,12104,2018-10-15,2,1,1


In [61]:
# Merge All three features dataframe

feature_merge = pd.merge(txn_data_group, txn_data_group_amt, how= 'inner', left_on='AgentRefID', right_on='AgentRefID')
txn_feature_merge = pd.merge(feature_merge, txn_data_group_date, how= 'inner', left_on='AgentRefID', right_on='AgentRefID')

txn_feature_merge.head()

,AgentRefID,total_payment,total_successfull_payments,total_failed_payments,total_amount_transaction,max_amount_transaction,min_amount_transaction,avg_amount_transaction,median_amount_transaction,total_amount_transaction_gt100,total_count_transaction_gt100,total_amount_transaction_gt1000,total_count_transaction_gt1000,total_amount_transcation_debit,total_amount_transaction_credit,total_unique_payee,total_unique_distributer,first_txn_date,last_payment_date,avg_count_payment_permonth,avg_count_payment_gt100_permonth,avg_amount_payment_permonth,avg_amout_payment_gt100_permonth,avg_last_month_amount_txn,avg_txn_first_week,avg_last_month_1week_amount_txn,last_month_amount_gt100
0,1,22,21,1,246.0,100.0,1.0,11.714286,2.0,0.0,0,0.0,0,238.0,8.0,8,1,2018-07-17,2018-12-20,4.0,2.000000,8.988889,100.000000,0.0,1.333333,0.0,0.0
1,55,132,128,4,1074317.0,25000.0,1.0,8393.101562,4999.0,1074274.0,85,1069574.0,79,1074274.0,43.0,52,1,2018-07-12,2018-09-06,56.0,28.333333,7712.888651,11878.973459,0.0,5923.153846,0.0,0.0
2,83,3361,3253,108,7335134.0,25000.0,1.0,2254.882877,1950.0,7333954.0,2320,6879216.0,1797,7334204.0,930.0,1573,1,2018-07-02,2019-01-21,133.5,387.000000,1848.848726,2784.038428,16004.0,2140.512927,0.0,16000.0
3,180,1965,1878,87,5929296.0,25000.0,1.0,3157.239617,2000.0,5929067.0,1847,5508229.0,1372,5929274.0,22.0,883,1,2018-07-02,2019-01-20,297.0,264.142857,2917.410086,2946.772422,33000.0,3118.101190,23500.0,33000.0
4,29,901,843,58,1891118.0,25000.0,1.0,2243.319098,2000.0,1890853.0,677,1799222.0,551,1890954.0,164.0,446,1,2018-07-02,2019-01-21,133.0,96.857143,2235.055844,2754.329794,178070.0,2148.735000,68861.0,178058.0


In [62]:
txn_feature_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4560 entries, 0 to 4559
Data columns (total 27 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   AgentRefID                        4560 non-null   int64         
 1   total_payment                     4560 non-null   int64         
 2   total_successfull_payments        4560 non-null   int64         
 3   total_failed_payments             4560 non-null   int64         
 4   total_amount_transaction          4560 non-null   float64       
 5   max_amount_transaction            4560 non-null   float64       
 6   min_amount_transaction            4560 non-null   float64       
 7   avg_amount_transaction            4550 non-null   float64       
 8   median_amount_transaction         4550 non-null   float64       
 9   total_amount_transaction_gt100    4560 non-null   float64       
 10  total_count_transaction_gt100     4560 non-null 

In [79]:
txn_feature_merge['vintage'] = (txn_feature_merge["last_payment_date"]-txn_feature_merge['first_txn_date']).dt.days

In [80]:
txn_feature_merge.head()

,AgentRefID,total_payment,total_successfull_payments,total_failed_payments,total_amount_transaction,max_amount_transaction,min_amount_transaction,avg_amount_transaction,median_amount_transaction,total_amount_transaction_gt100,total_count_transaction_gt100,total_amount_transaction_gt1000,total_count_transaction_gt1000,total_amount_transcation_debit,total_amount_transaction_credit,total_unique_payee,total_unique_distributer,first_txn_date,last_payment_date,avg_count_payment_permonth,avg_count_payment_gt100_permonth,avg_amount_payment_permonth,avg_amout_payment_gt100_permonth,avg_last_month_amount_txn,avg_txn_first_week,avg_last_month_1week_amount_txn,last_month_amount_gt100,vintage
0,1,22,21,1,246.0,100.0,1.0,11.714286,2.0,0.0,0,0.0,0,238.0,8.0,8,1,2018-07-17,2018-12-20,4.0,2.000000,8.988889,100.000000,0.0,1.333333,0.0,0.0,156.0
1,55,132,128,4,1074317.0,25000.0,1.0,8393.101562,4999.0,1074274.0,85,1069574.0,79,1074274.0,43.0,52,1,2018-07-12,2018-09-06,56.0,28.333333,7712.888651,11878.973459,0.0,5923.153846,0.0,0.0,56.0
2,83,3361,3253,108,7335134.0,25000.0,1.0,2254.882877,1950.0,7333954.0,2320,6879216.0,1797,7334204.0,930.0,1573,1,2018-07-02,2019-01-21,133.5,387.000000,1848.848726,2784.038428,16004.0,2140.512927,0.0,16000.0,203.0
3,180,1965,1878,87,5929296.0,25000.0,1.0,3157.239617,2000.0,5929067.0,1847,5508229.0,1372,5929274.0,22.0,883,1,2018-07-02,2019-01-20,297.0,264.142857,2917.410086,2946.772422,33000.0,3118.101190,23500.0,33000.0,202.0
4,29,901,843,58,1891118.0,25000.0,1.0,2243.319098,2000.0,1890853.0,677,1799222.0,551,1890954.0,164.0,446,1,2018-07-02,2019-01-21,133.0,96.857143,2235.055844,2754.329794,178070.0,2148.735000,68861.0,178058.0,203.0


In [85]:
import datetime
date = datetime.datetime(2019, 1, 22, 0, 0)

txn_feature_merge['recency'] = (date - txn_feature_merge["last_payment_date"]).dt.days

In [87]:
txn_feature_merge.drop(["last_payment_date", "first_txn_date"], axis=1, inplace=True)

In [88]:
# Work on ticket_data

In [89]:
ticket_data.shape

(25574, 2)

In [90]:
ticket_data.head()

,TicketCreatedDateTime,AgentRefID
0,2018-11-12,3
1,2018-11-12,3
2,2018-09-19,4
3,2018-08-30,4
4,2018-09-14,4


In [91]:
# Features from PayRefID and CreatedDate
# Features created from amount column
def get_consolidated_date_ticket_data(ticket_details):
    
    # total_ticket_raised
    total_ticket_raised  = len(ticket_details)
   
    
    ticket_details['Year'] = ticket_details["TicketCreatedDateTime"].dt.year
    ticket_details['Month'] = ticket_details["TicketCreatedDateTime"].dt.month
    ticket_details['Days'] = ticket_details["TicketCreatedDateTime"].dt.day
    
    # total unique payee
    total_ticket_raised_last_2_month = (ticket_details[ticket_details.Month.isin([12,1])]["AgentRefID"].count())
    avg_ticked_rasied_per_month = (ticket_details.groupby(["Year", "Month"])["AgentRefID"].count()).mean()
    
    return total_ticket_raised, total_ticket_raised_last_2_month, avg_ticked_rasied_per_month

In [92]:
ticket_data_group= ticket_data.groupby("AgentRefID", sort=False).apply(lambda cust: get_consolidated_date_ticket_data(cust))

In [93]:
ticket_data_group = ticket_data_group.reset_index()
ticket_data_group[['total_ticket_raised', 'total_ticket_raised_last_2_month', 'avg_ticked_rasied_per_month']] = pd.DataFrame(ticket_data_group[0].tolist(), index=ticket_data_group.index)
ticket_data_group = ticket_data_group.drop(0, axis=1)

In [94]:
ticket_data_group.head()

,AgentRefID,total_ticket_raised,total_ticket_raised_last_2_month,avg_ticked_rasied_per_month
0,3,2,0,2.000000
1,4,5,0,1.666667
2,12,51,2,10.200000
3,13,1,0,1.000000
4,14,3,2,1.500000


In [95]:
# Merge ticket and txn_data for final features set

In [96]:
features_df = pd.merge(txn_feature_merge, ticket_data_group, how= 'left', left_on='AgentRefID', right_on='AgentRefID')

In [97]:
features_df.shape

(4560, 30)

In [98]:
# checking missing values
missing(features_df)

avg_ticked_rasied_per_month         40.70
total_ticket_raised                 40.70
total_ticket_raised_last_2_month    40.70
avg_txn_first_week                  14.19
avg_count_payment_gt100_permonth     2.19
avg_amout_payment_gt100_permonth     2.19
avg_amount_payment_permonth          0.22
vintage                              0.22
avg_count_payment_permonth           0.22
recency                              0.22
avg_amount_transaction               0.22
median_amount_transaction            0.22
total_successfull_payments           0.00
total_payment                        0.00
total_failed_payments                0.00
total_amount_transaction_gt100       0.00
total_amount_transaction             0.00
max_amount_transaction               0.00
min_amount_transaction               0.00
total_amount_transaction_credit      0.00
total_count_transaction_gt100        0.00
total_amount_transaction_gt1000      0.00
total_count_transaction_gt1000       0.00
total_amount_transcation_debit    

In [99]:
# missing values imputed with 0 

features_df.fillna(0, inplace=True)

In [100]:
# label Creation

In [101]:
label_data_group = label_data.groupby(["AgentRefID"])["Amount"].count().reset_index()

In [102]:
label_data_group.shape

(2934, 2)

In [103]:
# 2934 are the ones who has transacted in last 1 month, So their Churn is equal to 0

label_data_group["Churn"]= 0

In [104]:
label_data_group = label_data_group[["AgentRefID","Churn"]]

In [105]:
label_data_group.head()

,AgentRefID,Churn
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [106]:
# Merge with final dataset to get the final dataframe for EDA and model training

churn_dataset = pd.merge(features_df, label_data_group, how= 'left', left_on='AgentRefID', right_on='AgentRefID')

In [107]:
missing(churn_dataset)

Churn                               36.45
total_amount_transaction_credit      0.00
total_payment                        0.00
total_successfull_payments           0.00
total_failed_payments                0.00
total_amount_transaction             0.00
max_amount_transaction               0.00
min_amount_transaction               0.00
avg_amount_transaction               0.00
median_amount_transaction            0.00
total_amount_transaction_gt100       0.00
total_count_transaction_gt100        0.00
total_amount_transaction_gt1000      0.00
total_count_transaction_gt1000       0.00
total_amount_transcation_debit       0.00
total_unique_payee                   0.00
avg_ticked_rasied_per_month          0.00
total_unique_distributer             0.00
avg_count_payment_permonth           0.00
avg_count_payment_gt100_permonth     0.00
avg_amount_payment_permonth          0.00
avg_amout_payment_gt100_permonth     0.00
avg_last_month_amount_txn            0.00
avg_txn_first_week                

In [108]:
# 36.45% of the customer is going to churn in next month

churn_dataset.fillna(1, inplace=True)

In [109]:
churn_dataset.head()

,AgentRefID,total_payment,total_successfull_payments,total_failed_payments,total_amount_transaction,max_amount_transaction,min_amount_transaction,avg_amount_transaction,median_amount_transaction,total_amount_transaction_gt100,total_count_transaction_gt100,total_amount_transaction_gt1000,total_count_transaction_gt1000,total_amount_transcation_debit,total_amount_transaction_credit,total_unique_payee,total_unique_distributer,avg_count_payment_permonth,avg_count_payment_gt100_permonth,avg_amount_payment_permonth,avg_amout_payment_gt100_permonth,avg_last_month_amount_txn,avg_txn_first_week,avg_last_month_1week_amount_txn,last_month_amount_gt100,vintage,recency,total_ticket_raised,total_ticket_raised_last_2_month,avg_ticked_rasied_per_month,Churn
0,1,22,21,1,246.0,100.0,1.0,11.714286,2.0,0.0,0,0.0,0,238.0,8.0,8,1,4.0,2.000000,8.988889,100.000000,0.0,1.333333,0.0,0.0,156.0,33.0,0.0,0.0,0.000000,0.0
1,55,132,128,4,1074317.0,25000.0,1.0,8393.101562,4999.0,1074274.0,85,1069574.0,79,1074274.0,43.0,52,1,56.0,28.333333,7712.888651,11878.973459,0.0,5923.153846,0.0,0.0,56.0,138.0,0.0,0.0,0.000000,1.0
2,83,3361,3253,108,7335134.0,25000.0,1.0,2254.882877,1950.0,7333954.0,2320,6879216.0,1797,7334204.0,930.0,1573,1,133.5,387.000000,1848.848726,2784.038428,16004.0,2140.512927,0.0,16000.0,203.0,1.0,1.0,0.0,1.000000,0.0
3,180,1965,1878,87,5929296.0,25000.0,1.0,3157.239617,2000.0,5929067.0,1847,5508229.0,1372,5929274.0,22.0,883,1,297.0,264.142857,2917.410086,2946.772422,33000.0,3118.101190,23500.0,33000.0,202.0,2.0,7.0,0.0,2.333333,0.0
4,29,901,843,58,1891118.0,25000.0,1.0,2243.319098,2000.0,1890853.0,677,1799222.0,551,1890954.0,164.0,446,1,133.0,96.857143,2235.055844,2754.329794,178070.0,2148.735000,68861.0,178058.0,203.0,1.0,2.0,0.0,1.000000,0.0


In [110]:
churn_dataset.to_csv("churn_dataset.csv", index= False)